# OpenFold Colab

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import py3Dmol
from proteome import protein
from proteome.models.folding.openfold import config
from proteome.models.folding.openfold.modeling import OpenFoldForFolding, OPENFOLD_MODEL_URLS
from proteome.models.folding.openfold.np.relax import relax

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


In [3]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [4]:
OPENFOLD_MODEL_URLS.keys()

dict_keys(['finetuning-3', 'finetuning-4', 'finetuning-5', 'finetuning_ptm-2', 'finetuning_no_templ_ptm-1'])

In [5]:
folder = OpenFoldForFolding(model_name="finetuning_ptm-2", random_seed=0)

In [5]:
#folder = OpenFoldForFolding(random_seed=0)
predicted_protein, plddt = folder.fold(sequence)

Running jackhmmer on uniref90 database...
Running jackhmmer on smallbfd database...
Running jackhmmer on mgnify database...
58 sequences found in uniref90
110 sequences found in smallbfd
9 sequences found in mgnify


In [6]:
unrelaxed_pdb_str = protein.to_pdb(predicted_protein)

In [7]:
amber_relaxer = relax.AmberRelaxation(
    max_iterations=0,
    tolerance=2.39,
    stiffness=10.0,
    exclude_residues=[],
    max_outer_iterations=20,
    use_gpu=False,
)
relaxed_pdb, _, _ = amber_relaxer.process(
    prot=predicted_protein, cif_output=False
)

In [8]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(relaxed_pdb)
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol